### Workshop 01 - TF Representation

#### FFT - The Frequency Fourier Transform

##### A. Physical Meaning  

##### B. Module and Phase 

##### C. Padding a Signal 

##### D. Simple Speech Enhancement - Pure Tone 

##### E. The Spectrogram 

##### F. Mel-Spectrogram and MFCC 

##### G. KWD

##### H. Music Recognition (Shazam)